<a href="https://colab.research.google.com/github/CPA-Dev/dezoomcamp/blob/main/Copy_of_Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
total = 0
for sqrt_value in generator:
    total += sqrt_value
    print(sqrt_value)

print(total)



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
%%capture
!pip install "dlt[duckdb]" # Install dlt with all the necessary DuckDB dependencies

In [ ]:
!dlt --version

dlt 0.4.3


In [ ]:
!pip install duckdb

In [ ]:
import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

# Create a DuckDB database

pipeline = dlt.pipeline(
    pipeline_name="append",
    destination="duckdb",
    dataset_name="append_db_new"
)

load_info = pipeline.run(people_1, table_name="people_new")

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_2,
					table_name="people_new",
					write_disposition="append"
					)

# show the outcome
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
display(conn.sql("DESCRIBE"))

stats_table = conn.sql("SELECT * FROM people_new").df()
display(stats_table)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


┌──────────┬───────────────┬─────────────────────┬──────────────────────┬──────────────────────────────────┬───────────┐
│ database │    schema     │        name         │     column_names     │           column_types           │ temporary │
│ varchar  │    varchar    │       varchar       │      varchar[]       │            varchar[]             │  boolean  │
├──────────┼───────────────┼─────────────────────┼──────────────────────┼──────────────────────────────────┼───────────┤
│ append   │ append_db     │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIGINT, TIM…  │ false     │
│ append   │ append_db     │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCHAR, VARC…  │ false     │
│ append   │ append_db     │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMESTAMP WIT…  │ false     │
│ append   │ append_db     │ people              │ [id, name, age, ci…  │ [BIGINT, VARCHAR, BIGINT, VARC…  │ false     │
│ append   │ append_db_new │ _dl

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707433625.4940073,ZScdWDxFixuqGw,None
1,2,Person_2,27,City_A,1707433625.4940073,n465Yn8QiRgPqw,None
2,3,Person_3,28,City_A,1707433625.4940073,l7CBjm6Zn9gZvA,None
3,4,Person_4,29,City_A,1707433625.4940073,9XmAz7hvfMkcOA,None
4,5,Person_5,30,City_A,1707433625.4940073,/HcbmICqA0QICg,None
5,3,Person_3,33,City_B,1707433626.1767018,vD0FgpKDP+cBqQ,Job_3
6,4,Person_4,34,City_B,1707433626.1767018,pEsJmGE+g61kig,Job_4
7,5,Person_5,35,City_B,1707433626.1767018,d3h6fBcasb+W4g,Job_5
8,6,Person_6,36,City_B,1707433626.1767018,PCAXO2i+hTPf3Q,Job_6
9,7,Person_7,37,City_B,1707433626.1767018,HRiPN5W09jh9RA,Job_7


In [ ]:
stats_table = conn.sql("SELECT sum(age) FROM people_new").df()
display(stats_table)

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]
!pip install duckdb

In [ ]:
# to do: homework :)

import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

# Create a DuckDB database

pipeline = dlt.pipeline(
    pipeline_name="merge",
    destination="duckdb",
    dataset_name="merge_db"
)

load_info = pipeline.run(people_1, table_name="people_new")

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


# run the pipeline with default settings, and capture the outcome
info = pipeline.run (people_2,
                    table_name="people_new",
                    write_disposition="merge",
                    primary_key="id"
					)

# show the outcome
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
display(conn.sql("DESCRIBE"))

all_table = conn.sql("SELECT * FROM people_new").df()
display(all_table)

stats_table = conn.sql("SELECT sum(age) FROM people_new").df()
display(stats_table)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


┌──────────┬──────────────────┬─────────────────────┬──────────────────────┬───────────────────────────────┬───────────┐
│ database │      schema      │        name         │     column_names     │         column_types          │ temporary │
│ varchar  │     varchar      │       varchar       │      varchar[]       │           varchar[]           │  boolean  │
├──────────┼──────────────────┼─────────────────────┼──────────────────────┼───────────────────────────────┼───────────┤
│ merge    │ merge_db         │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIGINT, …  │ false     │
│ merge    │ merge_db         │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCHAR, V…  │ false     │
│ merge    │ merge_db         │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMESTAMP …  │ false     │
│ merge    │ merge_db         │ people              │ [id, name, age, ci…  │ [BIGINT, VARCHAR, BIGINT, V…  │ false     │
│ merge    │ merge_db         │ 

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707435107.4863496,8yT5jmkdMx1/0Q,None
1,2,Person_2,27,City_A,1707435107.4863496,obB+Ck2igKkl7w,None
2,5,Person_5,35,City_B,1707435108.161965,w1taTIRoCQXEpg,Job_5
3,8,Person_8,38,City_B,1707435108.161965,M1b3zq0lQJ8lgA,Job_8
4,7,Person_7,37,City_B,1707435108.161965,BQHUP1xujFwO0A,Job_7
5,4,Person_4,34,City_B,1707435108.161965,lRyYputmdY6hag,Job_4
6,3,Person_3,33,City_B,1707435108.161965,iO+aOgEFvn1mLg,Job_3
7,6,Person_6,36,City_B,1707435108.161965,6u6g4OqWJnrUfA,Job_6


,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX